# How to optimize scientific python code

This notebook gives tips to optimize pure python and Numpy codes. This is mainly for scientific codes so strings or dictionnaries will not be covered in this document but infos about those topics can be found in the references and the end of the notebook.

**Table of content**
1. [Finding the bottlenecks by profiling](#profile)
2. [Pure python code](#pure)
3. [Numpy code](#numpy)
4. [References](#ref)
5. [Further readings](#further)


## Finding the bottlenecks by profiling <a name="profile"></a>

Firstly, note that optimizing the whole code is not a good idea since it requires a lot of time even where the gain is not worth it. Indeed, you just need to optimize the slow parts of your code. In this situation, you can optimize those parts or write them in compiled code (cython, C, ...) but this notebook will focus on the first option because in some case optimizing the python code can achieve a sufficient speed up without rewriting it in a compiled language.


In this notebook, we will use the profiler cProfile that is not the most heavy in terms of overheads but also the line by line profile line_profiler.

In [this towards data science blog](https://towardsdatascience.com/how-to-profile-your-code-in-python-e70c834fad89), a decorator has been given to profile a function and find the bottlenecks:

In [ ]:
import cProfile
import pstats
from functools import wraps


def profile(output_file=None, sort_by='cumulative', lines_to_print=None, strip_dirs=False):
    """A time profiler decorator.
    Inspired by and modified the profile decorator of Giampaolo Rodola:
    http://code.activestate.com/recipes/577817-profile-decorator/
    Args:
        output_file: str or None. Default is None
            Path of the output file. If only name of the file is given, it's
            saved in the current directory.
            If it's None, the name of the decorated function is used.
        sort_by: str or SortKey enum or tuple/list of str/SortKey enum
            Sorting criteria for the Stats object.
            For a list of valid string and SortKey refer to:
            https://docs.python.org/3/library/profile.html#pstats.Stats.sort_stats
        lines_to_print: int or None
            Number of lines to print. Default (None) is for all the lines.
            This is useful in reducing the size of the printout, especially
            that sorting by 'cumulative', the time consuming operations
            are printed toward the top of the file.
        strip_dirs: bool
            Whether to remove the leading path info from file names.
            This is also useful in reducing the size of the printout
    Returns:
        Profile of the decorated function
    """

    def inner(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            _output_file = output_file or func.__name__ + '.prof'
            pr = cProfile.Profile()
            pr.enable()
            retval = func(*args, **kwargs)
            pr.disable()
            pr.dump_stats(_output_file)

            with open(_output_file, 'w') as f:
                ps = pstats.Stats(pr, stream=f)
                if strip_dirs:
                    ps.strip_dirs()
                if isinstance(sort_by, (tuple, list)):
                    ps.sort_stats(*sort_by)
                else:
                    ps.sort_stats(sort_by)
                ps.print_stats(lines_to_print)
            return retval

        return wrapper

    return inner


@profile(sort_by='cumulative', lines_to_print=10, strip_dirs=True)
def function(x):
    res = []
    double = lambda y: 2*y
    for i in range(len(x)):
        res.append(double(x[i]))

List = [1]*10_000_000
function(List)

In the output file, we can find this:

![cProfiler example](cProf.PNG)


The profiler has given the execution time for each function call and we can see that on the total of 4.126 s the most of the operation come from the operations in `function` (probably the for loop), the calls to `append` and the calls to `double`, the lambda function.

But we can also use a line by line profiler as `line_profiler`:

In [7]:
from line_profiler import
# from memory_profiler import profile   # for memory profiling
import atexit

profile = LineProfiler()
atexit.register(profile.print_stats)


@profile
def function2(x):
    res = []
    double = lambda y: 2*y
    for i in range(len(x)):
        res.append(double(x[i]))
    return 0


List = [1]*100_000
function2(List)
profile.print_stats()   # in IPython atexit.register() does not work so I need to add it manually or create a decorator to dot it

Timer unit: 1e-07 s

Total time: 0.156559 s
File: <ipython-input-7-20893b078624>
Function: function2 at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def function2(x):
    10         1         20.0     20.0      0.0      res = []
    11         1         13.0     13.0      0.0      double = lambda y: 2*y
    12    100001     510238.0      5.1     32.6      for i in range(len(x)):
    13    100000    1055309.0     10.6     67.4          res.append(double(x[i]))
    14         1          8.0      8.0      0.0      return 0



Thanks to this line by line profiler we clrealy see at which line(s) the code is slowing down, here it's the line in the for loop. 


To sum, up we have two tools to profile our code and find the bottlenecks that have to be optimized.

## Pure python code <a name="pure"></a>

### Loops

For loops are costly in python... thus one can propose to use the `map` function to move the for loop into the C code and not python code anymore.


In [13]:
myList = ["a", "B", "c"]*1000

def f1(l1):
    res = []
    for i in range(len(l1)):
        res.append(l1[i].upper())
    return res

def f2(l2):
    return map(str.upper, l2)


%timeit f1(myList)
%timeit f2(myList)

608 µs ± 74.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
284 ns ± 23.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Ok so map is clrealy much faster than a for loop but is it always the case?

In [29]:
import timeit


xs = list(range(10))

def f1(l1):
    return list(map(lambda x: x*x, l1))

def f2(l2):
    return [x*x for x in l2]

def f3(l3):
    res = [0]*len(l3)
    for i in range(len(l3)):
        res[i] = l3[i]*l3[i]
    return res

def f4(l4):
    return (x*x for x in l4)

def f5(l5):
    return map(lambda x: x*x, l5)


%timeit f1(xs)
%timeit f2(xs)
%timeit f3(xs)
%timeit f4(xs)
%timeit f5(xs)

1.43 µs ± 72.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
733 ns ± 106 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.57 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
357 ns ± 7.57 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
314 ns ± 44.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Oops... in this case, `map` is clearly slower than a list comprehension (function `f2`) when we need to build a list...

In fact, in python the function overheads are heavy and creating a lambda or a python function via `def` can slow down the map function.

For the two lasts functions, this makes a generator that is clrealy much faster than a list and is lazy, so that each element is generated only when it's required (so no memory problems). And in this case, the map function becomes faster but the difference is not as high as we can expect for a Python VS C code performance comparison.

As a general tip, try and timeit because a list comprehension can be faster than map especially if you map a python non built in function.


### Avoid dots

In [38]:
def f1(list):
    newlist = []
    append = newlist.append
    doHash = int.__hash__
    for element in list:
        append(doHash(element))

def f2(list):
    newlist = []
    for element in list:
        newlist.append(int.__hash__(element))


%timeit f1([2]*200)
%timeit f2([2]*200)

31.2 µs ± 4.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
42.9 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Even if we use `append` (which is not recommended for optimizations) we can observe a speed up with those simple changes.

However, this makes the code a harder to read and maintain therefore you should be very familiar with your code when you use this.

### Local variables

In [44]:
def f(x):
    return x+1

def g(x):
    y = []
    for i in range(len(x)):
        y.append(f(x[i]))

def h(x):
    y = []
    function = f
    append = y.append
    for i in range(len(x)):
        append(function(x[i]))

def h2(x):
    y = []
    function = f
    for i in range(len(x)):
        y.append(function(x[i]))

%timeit g([1]*100)
%timeit h([1]*100)
%timeit h2([1]*100)

20.3 µs ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15.4 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
18.7 µs ± 983 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In this code, `h` is the most optimized function by putting `f` as a local variable as well as `y.append` therefore it achieves an intersting speed up compared to the non optimized function `g` and semi optimized function `h2` with only 100 elements.


In the following code, it's shown that we need to access a global variable as little as possible, as already mentionned: use local variables.

In [49]:
x1 = 0
x2 = 0

def f(x):
    global x1
    for elem in x:
        x1 += elem


def g(x):
    global x2
    res = 0
    for elem in x:
        res += elem
    x2 += res


%timeit f([1]*100)
%timeit g([1]*100)

7 µs ± 333 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.81 µs ± 609 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Data aggregation

The idea is to call function as little as possible since it has a high overhead. Therefore, prepare the data before the function and if possible call the function on a X-sized data set rather than call X times the function on each element (the code is from https://wiki.python.org/moin/PythonSpeed/PerformanceTips).

In [46]:
import time
x = 0
def doit1(i):
    global x
    x = x + i

list = range(100000)
t = time.time()
for i in list:
    doit1(i)

print("%.3f" % (time.time()-t))

0.026


In [47]:
import time
x = 0
def doit2(list):
    global x
    for i in list:
        x = x + i

list = range(100000)
t = time.time()
doit2(list)
print("%.3f" % (time.time()-t))

0.009


### Re map functions at runtime
The code of this section are also taken from the wiki python. The idea is to avoid the if statement that only check if it's called for the first time. We can directly redefined the function after the first call and there are no more `if/else`. 

In [53]:
 class Test:
    def check(self,a,b,c):
        if a == 0:
            self.str = b*100
        else:
            self.str = c*100

a = Test()
def example():
    for i in range(0,100000):
        a.check(i,"b","c")

%timeit example()

32.3 ms ± 5.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [54]:
class Test2:
    def check(self,a,b,c):
        self.str = b*100
        self.check = self.check_post
    def check_post(self,a,b,c):
        self.str = c*100

a = Test2()
def example2():
    for i in range(0,100000):
        a.check(i,"b","c")

%timeit example2()

25.6 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Final tips for this part

* Move computations as much as you can outside of the loops
* Same for memory allocation

## Numpy code <a name="numpy"></a>

### Use vectorization

Indeed vectorization can replace loops and bring the code in the C part to achieve huge speed ups. To do this you need to be comfortable with the indexing notation whose basic elements can be found below:

In [89]:
import numpy as np

A = np.random.rand(10,12)
v = np.array([1,2])

print("Shape of A[:2,:] =", A[:2,:].shape)  # those are the first two rows
print("Shape of A[:,1:3] =", A[:, 1:3].shape)  # those are the column 1 and 2 so the second and third one in 0 based indexing

print(f"Shape of A[::2,1] = {A[::2,:].shape}")  # take one row out of two so those are rows 0,2,4,6,8

print(f"v[::-1] = {v[::-1]}")   # reverse the array v

Shape of A[:2,:] = (2, 12)
Shape of A[:,1:3] = (10, 2)
Shape of A[::2,1] = (5, 12)
v[::-1] = [2 1]


**Tip**: if you have a function that you want to apply to numpy arrays, think about np.vectorize to vectorize automatically your function (avoid loops). The following example is taken from the documentation.

In [81]:
import numpy as np

def myfunc(a, b):   # you just need to create an element function that will be applied elementwise
    "Return a-b if a>b, otherwise return a+b"
    if a > b:
        return a - b
    else:
        return a + b

vfunc = np.vectorize(myfunc)
print(vfunc([1, 2, 3, 4], 2))
print(vfunc([1, 2, 3, 4], [2,1,1,2]))

[3 4 1 2]
[3 1 2 2]


### Local variables

Again, use local variables with arrays to avoid creation of temporary arrays that will slow down the code:

In [85]:
import numpy as np


def f(x):
    return 2*x**2 + 3*x - 4*x**3

def g(x):
    gx = 2*x**2
    gx += 3*x
    gx -= 4*x**3
    return gx

%timeit f(np.random.rand(10000))
%timeit g(np.random.rand(10000))


500 µs ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
492 µs ± 3.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Note that the speed up can be more important according to the computations inside the function. 

### Broadcasting

Using broacast allows us to make computations on smaller arrays and then combining them with arrays of the same shape or not.

From the doc:
> When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing (i.e. rightmost) dimensions and works its way left. Two dimensions are compatible when

> 1. they are equal, or

> 2. one of them is 1

In [88]:
A = np.array([[1,2,],[1,2]])
v = np.array([1,2])

print(A/v[:,None])  # divide columnwise such that A[0,:]/v[0] and A[1,:]/v[1]
print(A/v)          # divide rowwise such that A[:,0]/v[0] and A[:,1]/v[1]
print(A/v[None,:])  # same

[[1.  2. ]
 [0.5 1. ]]
[[1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]]


Note that in the above example, we have not increase the size of `v` to make it work!


The example of outer product by broadcasting:

In [91]:
x = np.array([1, 2, 3, 4], dtype=np.int16)
y = np.array([5, 6, 7], dtype=np.int16)
print(x[np.newaxis,:] * y[:,np.newaxis])   
print(x[None, :]* y[:,None])
print(x[np.newaxis,:])
print(y[:,np.newaxis])


[[ 5 10 15 20]
 [ 6 12 18 24]
 [ 7 14 21 28]]
[[ 5 10 15 20]
 [ 6 12 18 24]
 [ 7 14 21 28]]
[[1 2 3 4]]
[[5]
 [6]
 [7]]


In fact, `x` will be of shape (1,4) and y (3,1) which will lead to (4,3). it will behaves like if `x` is copied 2 times below it to form a (3,4) matrix and `y` 3 times at its right side to form a (3,4) matrix and then we apply a **element wise** matrix product.

### In place opeartions

In place operations are much faster and in the following code, we can see that a small change in the code gives us an intersting speed up.

In [57]:
import numpy as np
a = np.zeros(10**7)
%timeit global a ; a = 0*a
%timeit global a ; a *= 0

40.9 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.9 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Avoid copies when possible

Indeed, copying an array is as costly as making an operation on it:

In [58]:
import numpy as np
a = np.zeros(10**7)
%timeit a.copy()
%timeit a + 1

41.5 ms ± 3.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.6 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Cache effects

If you need to go through your array, try to access it in a continuous way and not a random one. In fact, big strides causes caches misses and longer access time.

It can be interesting to choose the ordering: row strorage (default `'C'`) or colmunwise (Fortran ordering `'F'`) according to the computations you have to do:

In [75]:
import numpy as np
c = np.zeros((10**4, 10**4), order='C')
f = np.zeros((10**4, 10**4), order='F')
%timeit c.sum(axis=0)
%timeit f.sum(axis=0)
%timeit c.sum(axis=1)
%timeit f.sum(axis=1)

print(c.strides)   # stride[0] is the number of bytes to skip to go to the next row
print(f.strides)   # stride[1] is the number of bytes to skip to go to the next column

96.6 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
146 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
138 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
91.3 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
(80000, 8)
(8, 80000)


Here you can see that 1 element is 8 bytes, so this can be intersting to reduce the size of each element if this is possible for example using type uint8 which is 1 byte big so 8x smaller than above. The strides will be smaller and accesses may be faster (this needs to be tried in your particular case).

Moreover, we can force the array (1D, 2D, 3D, ...) to be contiguous in memory which is faster for computations in some cases, you can use numpy.ascontiguousarray to do it.

### Masks

Any time you need to perform operations onto an array by ignoring some elements, think abouts masks which is an efficient way to do computations on a part of an array (avoid for loops).

But **not all NumPy functions respect masks**, for instance np.dot, so check the return types!

In [107]:
import numpy as np

x = np.array([1, 2, 3, -99, 5])

maskedX = np.ma.masked_array(x, mask=[0, 0, 0, 1, 0])

print(f"The type of a masked array is {type(maskedX)} not a {type(x)}")

print(f"Computation like np.mean will ignore the masked elements: np.mean(maskedX) = {np.mean(maskedX)}")

maskedX[1] = 9
print(f"But after having done maskedX[1] = 9, x will also be changed! Indeed, x[1] = {x[1]}")  # YOU CAN ALSO CHANGE THE MASKED VALUES

maskedX[1] = np.ma.masked  # make the value at index 1 masked
print("maskedX:", maskedX)

maskedX[1] = 9  # clear the mask by assignment
print("maskedX:", maskedX)

print(f"We can get the mask with maskedX;mask which gives us {maskedX.mask}")

print(f"We can fill the masked values with for example maskedX.filled(-1) that gives us {maskedX.filled(-1)}")


The type of a masked array is <class 'numpy.ma.core.MaskedArray'> not a <class 'numpy.ndarray'>
Computation like np.mean will ignore the masked elements: np.mean(maskedX) = 2.75
But after having done maskedX[1] = 9, x will also be changed! Indeed, x[1] = 9
maskedX: [1 -- 3 -- 5]
maskedX: [1 9 3 -- 5]
We can get the mask with maskedX;mask which gives us [False False False  True False]
We can fill the masked values with for example maskedX.filled(-1) that gives us [ 1  9  3 -1  5]


## References <a name="ref"></a>

More info about strings, list sorting and dictionaries: https://wiki.python.org/moin/PythonSpeed/PerformanceTips

More info about loop otpimization: https://www.python.org/doc/essays/list2str/

More info about numpy arrays in memory: https://hal.inria.fr/inria-00564007/en

More info abut advanced numpy: https://scipy-lectures.org/advanced/advanced_numpy/index.html#advanced-numpy

## Further readings: <a name="further"></a>

The `numexpr` is a fast numerical expression evaluator for NumPy.

The `numba` package allows jit compilation and even GPU acceleration with decorators.

`cython` allows to use compiled code and/or make a bridge between python and C codes.